In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/meghabhairi114/End-To-End-DSProject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="meghabhairi114"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b29b387dc277f7e7950d37ae3110a3626be2fb07"

In [2]:
%pwd

'c:\\Users\\HP\\Desktop\\MLFlow\\End-To-End-DSProject\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\HP\\Desktop\\MLFlow\\End-To-End-DSProject'

In [4]:
from  dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_filename: Path
    target_column: str
    mlflow_uri: str
    

In [5]:
from src.datascience.constants import CONFIG_FILEPATH, PARAMS_FILEPATH,SCHEMA_FILEPATH
from src.datascience.utils.common import read_yaml , create_directories , save_json

[2025-06-04 13:33:28,132: INFO: __init__: Logger initialized successfully in datascience package.]


In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILEPATH,
                 params_filepath=PARAMS_FILEPATH,
                 schema_filepath=SCHEMA_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
                root_dir=config.root_dir,
                test_data_path=config.test_data_path,
                model_path=config.model_path,
                all_params=params,
                metric_filename= config.metric_filename,
                target_column= schema.name,
                mlflow_uri="https://dagshub.com/meghabhairi114/End-To-End-DSProject.mlflow")
            
            
        return  model_evaluation_config

In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error , mean_absolute_error , r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [10]:
class ModelEvaluation:
    def __init__(self , config=ModelEvaluationConfig):
        self.config=config
        
    def eval_metrics(self , actual , pred):
        rmse=np.sqrt(mean_squared_error(actual  ,pred))
        mae= mean_absolute_error(actual , pred)
        r2 = r2_score(actual , pred)
        return rmse, mae, r2
    
    def  log_into_mlflow(self):
        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        test_x=test_data.drop([self.config.target_column] , axis=1)
        test_y=test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualitites=model.predict(test_x)

            (rmse , mae, r2)=self.eval_metrics(test_y , predicted_qualitites)

            #Save metrics
            scores={"rmse": rmse,"mae": mae ,"r2": r2}
            save_json(path=Path(self.config.metric_filename) ,data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse" , rmse)
            mlflow.log_metric("mae" , mae)
            mlflow.log_metric("r2" , r2)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-06-04 13:34:32,532: INFO: common: yaml file:config\config.yaml is loaded successfully]
[2025-06-04 13:34:32,538: INFO: common: yaml file:params.yaml is loaded successfully]
[2025-06-04 13:34:32,547: INFO: common: yaml file:schema.yaml is loaded successfully]
[2025-06-04 13:34:32,552: INFO: common: Created directory at :artifacts_root]
[2025-06-04 13:34:32,555: INFO: common: Created directory at :artifacts/model_evaluation]
[2025-06-04 13:34:32,970: INFO: common: json file saved at artifacts\model_evaluation\metrics.json]


2025/06/04 13:34:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2025/06/04 13:35:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


🏃 View run nimble-croc-635 at: https://dagshub.com/meghabhairi114/End-To-End-DSProject.mlflow/#/experiments/0/runs/cde98c3e63594547876085b716b240eb
🧪 View experiment at: https://dagshub.com/meghabhairi114/End-To-End-DSProject.mlflow/#/experiments/0
